In [ ]:
## Run the following scripts in terminal to obtain the bottle_necks for the training data

"python retrain.py --image_dir ~/dogs/train"  ## Retrain on Inception_V3 to obtain bottle necks as tmp file

"sudo cp -r /tmp/bottleneck/. ~/bottleneck"   ## copy bottle_neck temp file to home directory

In [47]:
## Importing required libraries

import numpy as np
import pandas as pd
from tqdm import tqdm
import cv2
import tensorflow as tf
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os as os
os.chdir('/home/himanshu/Dog_breeds_data')

In [6]:
labels = pd.read_csv("labels.csv")
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [12]:
## Reading bottle neck files as numpy array to create training dataset

os.chdir('/home/himanshu/')
bottleneck = []
for f, breed in tqdm(labels.values):
    file = open("bottleneck/"+'%s/'%(breed)+str(f)+".jpg_https~tfhub.dev~google~imagenet~inception_v3~feature_vector~1.txt")
    for line in file:
        currentline = line.split(",")
        arr = np.array(currentline)
        bottleneck.append(arr)

100%|██████████| 10222/10222 [00:01<00:00, 5303.61it/s]


In [38]:
## Label Encoding to obtain the dependent variable(Breed classes)

lab_enc = preprocessing.LabelEncoder()
lab_enc.fit(labels['breed'])
target = lab_enc.transform(labels['breed']) 
target.shape

(10222,)

In [24]:
data = pd.DataFrame(bottleneck,dtype='float64')

In [25]:
data.shape

(10222, 2048)

In [33]:
x_train, x_cv, y_train, y_cv = train_test_split(data, target, test_size = 0.2, random_state = 42)

### ML model training : Linear SVM Classification

In [39]:
from sklearn.svm import LinearSVC 

model = LinearSVC()
model.fit(x_train, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [40]:
coeffs = model.coef_
bias = model.intercept_

### Cross-Validating the model

In [42]:
y_pred = model.predict(x_cv)

In [48]:
accuracy_score(y_cv,y_pred)

0.88801955990220049